In [1]:
import os
import sys
import time
from pathlib import Path

import h5py
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from PIL import Image

from col_dtypes import ColDataTypes

In [2]:
data_folder = "../../data/" # root data folder
dpi = 50 # spectrogram image resolution

chunk_id = 3 # chunk 1 is noise, chunks 2-6 are earthquake signals
assert chunk_id > 0 and chunk_id <= 6

In [3]:
col_dtypes = ColDataTypes()
dtypes = col_dtypes.get_initial_dtype_dict()
date_cols = col_dtypes.get_date_cols()

csv_pth = os.path.join(data_folder, f'raw/chunk{chunk_id}/chunk{chunk_id}.csv') # chunk metadata
eqpath = os.path.join(data_folder, f'raw/chunk{chunk_id}/chunk{chunk_id}.hdf5') # chunk earthquake data

chunk = pd.read_csv(csv_pth, dtype=dtypes, parse_dates=date_cols, encoding='utf-8')
print(eqpath)

# processed_csv_save_pth = os.path.join(data_folder, f'raw/chunk{chunk_id}/chunk{chunk_id}_processed.csv') # processed metadata save pth

chunk.columns

../../data/raw/chunk3/chunk3.hdf5


Index(['network_code', 'receiver_code', 'receiver_type', 'receiver_latitude',
       'receiver_longitude', 'receiver_elevation_m', 'p_arrival_sample',
       'p_status', 'p_weight', 'p_travel_sec', 's_arrival_sample', 's_status',
       's_weight', 'source_id', 'source_origin_time',
       'source_origin_uncertainty_sec', 'source_latitude', 'source_longitude',
       'source_error_sec', 'source_gap_deg',
       'source_horizontal_uncertainty_km', 'source_depth_km',
       'source_depth_uncertainty_km', 'source_magnitude',
       'source_magnitude_type', 'source_magnitude_author',
       'source_mechanism_strike_dip_rake', 'source_distance_deg',
       'source_distance_km', 'back_azimuth_deg', 'snr_db', 'coda_end_sample',
       'trace_start_time', 'trace_category', 'trace_name'],
      dtype='object')

# Data Cleaning

In [4]:
# Get chunk w relevent columns
arrival_df = chunk
# Convert 'coda_end_sample' to int
arrival_df.loc[:, ['coda_end_sample']] = arrival_df['coda_end_sample'].apply(lambda x: int(x[2:-3]))
arrival_df.astype({
    'p_arrival_sample': 'int16',
    's_arrival_sample': 'int16',
    })
# Check if 'p_status' == 's_status'
arrival_df.loc[:, ['same_status?']] = arrival_df[['p_status', 's_status']].apply(lambda x: x['p_status'] == x['s_status'], axis=1)

# New columns

In [5]:
arrival_df.loc[:, ['p_duration']] = arrival_df.apply(lambda col: col['s_arrival_sample'] - col['p_arrival_sample'], axis=1)
arrival_df.loc[:, ['s_duration']] = arrival_df.apply(lambda col: col['coda_end_sample'] - col['s_arrival_sample'], axis=1)

In [6]:
# Find min/max p and s wave durations
arrival_df[['p_duration', 's_duration']].describe()

,p_duration,s_duration
count,200000.000000,200000.000000
mean,646.533370,1056.422661
std,488.293678,638.257479
min,2.000000,9.000000
25%,288.000000,623.000000
50%,507.000000,864.000000
75%,888.000000,1306.000000
max,4914.000000,5010.000000


In [7]:
# only accept waveforms that satisfy 'min_acceptable_duration' 
min_acceptable_duration = 500 # 100 == 1 second
noise_buffer = 100

P_arrival_df = arrival_df.loc[arrival_df['p_duration'] >= min_acceptable_duration, :].reset_index(drop=True)
S_arrival_df = arrival_df.loc[arrival_df['s_duration'] >= min_acceptable_duration, :].reset_index(drop=True)
noise_df = arrival_df.loc[arrival_df['p_arrival_sample'] > min_acceptable_duration + noise_buffer, :].reset_index(drop=True)

print('P-waves')
display(P_arrival_df.loc[:, ['p_duration', 's_duration']])
print('S-waves')
display(S_arrival_df.loc[:, ['p_duration', 's_duration']])
print('Noise:')
display(noise_df.loc[:, ['p_duration', 's_duration']])

P-waves


,p_duration,s_duration
0,855.0,828.0
1,858.0,824.0
2,849.0,673.0
3,1047.0,801.0
4,510.0,612.0
...,...,...
101241,763.0,1936.0
101242,1320.0,1090.0
101243,1796.0,793.0
101244,1060.0,1739.0


S-waves


,p_duration,s_duration
0,106.0,717.0
1,109.0,927.0
2,104.0,1111.0
3,855.0,828.0
4,858.0,824.0
...,...,...
180103,763.0,1936.0
180104,1320.0,1090.0
180105,1796.0,793.0
180106,1060.0,1739.0


Noise:


,p_duration,s_duration
0,147.0,455.0
1,163.0,496.0
2,104.0,1111.0
3,260.0,475.0
4,855.0,828.0
...,...,...
104209,1310.0,701.0
104210,1306.0,706.0
104211,411.0,588.0
104212,1320.0,1090.0


In [8]:
# Save resultant df
P_arrival_df.to_csv(os.path.join(data_folder, f'raw/chunk{chunk_id}/chunk{chunk_id}_P.csv'), index=False)
S_arrival_df.to_csv(os.path.join(data_folder, f'raw/chunk{chunk_id}/chunk{chunk_id}_S.csv'), index=False)
noise_df.to_csv(os.path.join(data_folder, f'raw/chunk{chunk_id}/chunk{chunk_id}_noise.csv'), index=False)

print('P-waves')
display(P_arrival_df.loc[:, ['p_arrival_sample', 's_arrival_sample', 'p_status', 's_status', 'coda_end_sample']])
print('S-waves')
display(S_arrival_df.loc[:, ['p_arrival_sample', 's_arrival_sample', 'p_status', 's_status', 'coda_end_sample']])
print('Noise:')
display(noise_df.loc[:, ['p_arrival_sample', 's_arrival_sample', 'p_status', 's_status', 'coda_end_sample']])

P-waves


,p_arrival_sample,s_arrival_sample,p_status,s_status,coda_end_sample
0,800.0,1655.0,manual,manual,2483
1,700.0,1558.0,automatic,automatic,2382
2,896.0,1745.0,autopicker,autopicker,2418
3,389.0,1436.0,autopicker,autopicker,2237
4,895.0,1405.0,autopicker,autopicker,2017
...,...,...,...,...,...
101241,500.0,1263.0,manual,manual,3199
101242,900.0,2220.0,automatic,automatic,3310
101243,800.0,2596.0,automatic,automatic,3389
101244,400.0,1460.0,manual,manual,3199


S-waves


,p_arrival_sample,s_arrival_sample,p_status,s_status,coda_end_sample
0,399.0,505.0,autopicker,autopicker,1222
1,600.0,709.0,manual,manual,1636
2,800.0,904.0,manual,manual,2015
3,800.0,1655.0,manual,manual,2483
4,700.0,1558.0,automatic,automatic,2382
...,...,...,...,...,...
180103,500.0,1263.0,manual,manual,3199
180104,900.0,2220.0,automatic,automatic,3310
180105,800.0,2596.0,automatic,automatic,3389
180106,400.0,1460.0,manual,manual,3199


Noise:


,p_arrival_sample,s_arrival_sample,p_status,s_status,coda_end_sample
0,900.0,1047.0,manual,manual,1502
1,700.0,863.0,manual,manual,1359
2,800.0,904.0,manual,manual,2015
3,800.0,1060.0,manual,manual,1535
4,800.0,1655.0,manual,manual,2483
...,...,...,...,...,...
104209,1000.0,2310.0,automatic,automatic,3011
104210,900.0,2206.0,manual,manual,2912
104211,900.0,1311.0,manual,manual,1899
104212,900.0,2220.0,automatic,automatic,3310


In [9]:
# from PIL import Image

# data_folder = "../../data/" # root data folder
# save_folder = data_folder + 'images/' # folder to save spectrogram images
# p_folder = save_folder + 'P/'
# fn = p_folder + "109C.TA_20060723155859_EV.png"
# np.array(Image.open(fn)).shape